In [1]:
# Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
from pyspark.sql import SparkSession
from pyspark.sql.types import *
%matplotlib inline

In [2]:
spark = SparkSession.builder.master("local").appName('Ops').getOrCreate()

In [3]:
path = "E:/Rutgers/Projects/MDSR/IPL-MSDR"

In [ ]:
# Reading Data
matches = pd.read_csv(path + '/dataset/original_ipldata/matches.csv')
deliveries = pd.read_csv(path + '/dataset/original_ipldata/deliveries.csv')

In [ ]:
# Schema of original data (matches.csv)
matches.info()

In [ ]:
# Schema of original data (deliveries.csv)
deliveries.info()

# Data Cleaning

In [ ]:
# Dropping columns that are of no use
matches = matches.drop(columns = ['umpire1', 'umpire2','umpire3','date'])

In [ ]:
#Filing empty values
matches = matches.fillna(value = 'None')
deliveries = deliveries.fillna(value = 0)

In [ ]:
# Schema of cleaned data (matches.csv)
matches.info()

In [ ]:
# Schema of cleaned data (deliveries.csv)
deliveries.info()

In [ ]:
# Saving cleaned data (matches.csv)
matches.to_csv(path + '/dataset/clean_data/matches.csv')

In [ ]:
# Saving cleaned data (deliveries.csv)
deliveries.to_csv(path + '/dataset/clean_data/deliveries.csv')

# Basic Analysis

In [ ]:
# Teams playing in the league
teams = matches['team1'].unique()
print("Total number of teams participated so far: " + str(len(matches['team1'].unique())))
print("Teams participated so far: ")
for i in teams:
    print("- " + i)

In [ ]:
# Total Venues
print("Number of venues matches were played: " + str(len(matches['venue'].unique())))
for i in matches['venue'].unique():
    print("- " + i)

In [ ]:
# Cities the matches were played
print("Number of cities matches were played: " + str(len(matches['city'].unique())))
for i in matches['city'].unique():
    print("- " + i)

In [ ]:
# Total number of bowlers so far
print("Total number of bowlers: " + str(len(deliveries['bowler'].unique())))

In [ ]:
# Total number of batsmen so far
print("Total number of batsmen: " + str(len(deliveries['batsman'].unique())))

In [ ]:
# Total number of participating players
players = set()
for i in range(len(deliveries['match_id'])):
    players.add(deliveries['bowler'][i])
    players.add(deliveries['batsman'][i])
    players.add(deliveries['non_striker'][i])
print("Total number of player: " + str(len(players)))

# Spark Analysis

In [ ]:
matches = spark.read.csv(path + '/dataset/clean_data/matches.csv',inferSchema=True,header=True)
deliveries = spark.read.csv(path + '/dataset/clean_data/deliveries.csv',inferSchema=True,header=True)

### Total number of matches per season

In [ ]:
matches.registerTempTable('seasons')
seasons = spark.sql('''Select distinct(season),count(*) as total_matches from seasons group by season ''') 
seasons.show()

In [ ]:
# Plot
fig, a = plt.subplots()
a = sns.barplot(x ="season", y="total_matches", data=seasons.toPandas(),palette='viridis')
a.set_xlabel('Season')
a.set_ylabel('Total Matches')
a.set_title('Number of matches in each season')

### Number of maches played by each team since season 1

In [ ]:
matches.registerTempTable('team')
team = spark.sql('''Select distinct(team), count(*) as total_matches from (Select team1 as team from team UNION ALL (select team2 as team from team)) group by team ''')
team.show()

In [ ]:
# Plot
fig, a = plt.subplots(figsize = (5,5))
a = sns.barplot(x ="total_matches", y="team", data=team.toPandas(), palette='viridis')
a.set_ylabel('Team')
a.set_xlabel('Total Matches')
a.set_title('Number of matches played by each team')

### Total season in which teams have played

In [ ]:
matches.registerTempTable('team_season')
team_season = spark.sql('''Select team1 as team, min(season) as first_season, max(season) as last_season, count(distinct(season)) as total_seasons from team_season group by team1 order by total_seasons desc''')
team_season.show()

### Total number of matches won by teams

In [ ]:
matches.registerTempTable('most_win')
most_win = spark.sql('''Select distinct(winner) as team, count(*) as total_matches from most_win where winner <>'None' group by winner order by total_matches ''')
most_win.show()

In [ ]:
# Plot
fig, a = plt.subplots(figsize = (5,5))
a = sns.barplot(x ="total_matches", y="team", data=most_win.toPandas(), palette='viridis')
a.set_ylabel('Team')
a.set_xlabel('Total Matches')
a.set_title('Number of matches won by each team')

### Total matches won by teams in each season

In [ ]:
matches.registerTempTable('most_win_by_season')
most_win_by_season = spark.sql('''Select season, winner as team, count(*) as total_matches_won from most_win_by_season where winner <> 'None' group by season, winner order by total_matches_won desc''')
most_win_by_season.show()

### Players with maximum man of the match awards 

In [ ]:
matches.registerTempTable('man_match')
man_match = spark.sql('''Select distinct(player_of_match), count(*) as total_matches from man_match group by player_of_match order by total_matches desc limit 10 ''')
man_match.show()

In [ ]:
# Plot
fig, a = plt.subplots(figsize = (5,5))
a = sns.barplot(x ="total_matches", y="player_of_match", data=man_match.toPandas(), palette='viridis')
a.set_xlabel('Total Matches')
a.set_ylabel('Player')
a.set_title('Number of times player won man of the match')

### Number of matches per Venue

In [ ]:
matches.registerTempTable('venue')
venue = spark.sql('''Select distinct(venue), count(*) as total_matches from venue group by venue''')
venue.show()

In [ ]:
# Plot
fig, a = plt.subplots(figsize = (10,20))
a = sns.barplot(x ="total_matches", y="venue", data=venue.toPandas(), palette='viridis')
a.set_ylabel('Venue')
a.set_xlabel('Total Matches')
a.set_title('Number of matches at each venue')

### Percentage toss decisions 

In [ ]:
matches.registerTempTable('toss')
toss = spark.sql('''Select distinct(toss_decision), ((count(toss_decision)*100)/ (select count(*) from toss)) as percentage_count from toss group by toss_decision''')
toss.show()

In [ ]:
# Plot
fig, a = plt.subplots(figsize = (5,5))
a = sns.barplot(x ="toss_decision", y="percentage_count", data=toss.toPandas(), palette='viridis')
a.set_ylabel('Percentage')
a.set_xlabel('Toss Decision')
a.set_title('Percentage Plot of toss_decision')

### Percentage of team winning the toss as well as the match

In [ ]:
matches.registerTempTable('toss_and_won')
matches.registerTempTable('toss_won_data')
toss_won_data = spark.sql('''Select t1.season, t1.total_matches, \
          t2.count_toss_and_won as count_toss_and_won, \
          (t2.count_toss_and_won / t1.total_matches * 100) as percent_toss_and_won from \
          (Select distinct(season),count(*) as total_matches from seasons group by season)t1 \
          left join (Select distinct(season), count(*) as count_toss_and_won from toss_and_won where toss_winner = winner group by season)t2 on t1.season = t2.season order by season''')
toss_won_data.show()

In [ ]:
# Plot
fig, a = plt.subplots(figsize = (10,5))
a = sns.barplot(x ="season", y="percent_toss_and_won", data=toss_won_data.toPandas(), palette='viridis')
a.set_ylabel('Percentage')
a.set_xlabel('Season')
a.set_title('Percentage Plot of Season and Toss_and_won')

### Percentage matches won by batting first 

In [ ]:
win_batting_first = spark.sql('''Select t1.season, t1.total_matches, \
          t2.win_batting_first as win_batting_first, \
          (t2.win_batting_first/ t1.total_matches * 100) as percent_win_batting_first from \
          (Select distinct(season),count(*) as total_matches from seasons group by season)t1 \
          left join (Select distinct(season), count(*) as win_batting_first from seasons where win_by_runs > 0  group by season)t2 on t1.season = t2.season order by season ''')
win_batting_first.show()

In [ ]:
# Plot
fig, a = plt.subplots(figsize = (10,5))
a = sns.barplot(x ="season", y="percent_win_batting_first", data=win_batting_first.toPandas(), palette='viridis')
a.set_ylabel('Percentage')
a.set_xlabel('Season')
a.set_title('Percentage Plot of Season and won by batting')

### Percentage matches won by fielding first

In [ ]:
win_bowling_first = spark.sql('''Select t1.season, t1.total_matches, \
          t2.win_bowling_first as win_bowling_first, \
          (t2.win_bowling_first/ t1.total_matches * 100) as percent_win_bowling_first from \
          (Select distinct(season),count(*) as total_matches from seasons group by season)t1 \
          left join (Select distinct(season), count(*) as win_bowling_first from seasons where win_by_wickets > 0  group by season)t2 on t1.season = t2.season order by season ''')
win_bowling_first.show()

In [ ]:
# Plot
fig, a = plt.subplots(figsize = (10,5))
a = sns.barplot(x ="season", y="percent_win_bowling_first", data=win_bowling_first.toPandas(), palette='viridis')
a.set_ylabel('Percentage')
a.set_xlabel('Season')
a.set_title('Percentage Plot of Season and won by wickets ')

# Final Analysis

In [4]:
# Reading data
matches = spark.read.csv(path + '/dataset/clean_data/matches.csv',inferSchema=True,header=True)
deliveries = spark.read.csv(path + '/dataset/clean_data/deliveries.csv',inferSchema=True,header=True)

In [5]:
# Creating temporary tables of the data
matches.registerTempTable('matches_db')
deliveries.registerTempTable('deliveries_db')

In [6]:
# Merging both the tables
merged_db = spark.sql('select m.*,d.* from matches_db as m inner join deliveries_db as d on m.id=d.match_id')
merged_db.registerTempTable('analysis_db')

## Batting Metrics 

In [ ]:
# nmba: no. of batsmen
# nm: no. of matches played by a batsman
# hha: hard hitting ability
# f: finisher
# fsa: fast scoring ability
# con: consistency
# rbw: running between wickets

In [ ]:
# Number of Batsmen
nmba = spark.sql('select count(distinct(batsman)) as No_of_Batsman from analysis_db')
nmba.show()

In [ ]:
# Number of Matches played by a batsmen
nm = spark.sql('select batsman, count(distinct(match_id)) as No_of_Matches \
                from analysis_db group by batsman')
nm.registerTempTable('no_of_matches_table')

### Hard Hitting Ability 

In [ ]:
# Hard Hitting Ability = (4*Fours + 6*Sixes)/Balls Played by Batsman
hha = spark.sql('select nmt.batsman as Batsman, nvl(t4.hard_hitting_ability,0) as \
                Hard_Hitting_Ability from \
                (select t1.batsman, (t1.fours*4 + t2.sixes*6)/t3.balls_played as hard_hitting_ability\
                from (select batsman,count(*) as fours from analysis_db where batsman_runs = 4 group by batsman) t1 \
                inner join  \
                (select batsman,count(*) as sixes from analysis_db where batsman_runs = 6 \
                group by batsman) t2 on t1.batsman=t2.batsman\
                inner join\
                (select batsman,count(*) as balls_played from analysis_db \
                group by batsman) t3 on t3.batsman=t1.batsman) t4 \
                right join no_of_matches_table nmt on t4.batsman = nmt.batsman')
hha.registerTempTable('hard_hitting_ability_table')

In [ ]:
hha = spark.sql('select rank() over (order by Hard_Hitting_Ability desc) as Rank, t1.* \
                  from hard_hitting_ability_table t1 \
                  inner join \
                  no_of_matches_table t2\
                  on t1.batsman = t2.batsman where no_of_matches>9')
hha.registerTempTable('hard_hitting_ability_rank')
hha.show(10)

In [ ]:
hha = spark.sql('select t1.*, (240-rank)/240 as Points from hard_hitting_ability_rank t1')
hha.registerTempTable('hard_hitting_ability_points')
hha.show(10)

In [ ]:
hha = spark.sql('select t1.*, Points*1.25 as Weight from hard_hitting_ability_points t1')
hha.registerTempTable('hard_hitting_ability_weights')
hha.show(10)

### Finisher 

In [ ]:
# Finisher = Not Out innings/Total Innings played
f = spark.sql('select t3.batsman as Batsman, t3.not_out_innings/t4.total_matches_played as Finisher from\
              (select t1.batsman, t1.matches_played-t2.number_of_times_out as not_out_innings from \
              (select batsman, count(distinct(match_id)) as matches_played from analysis_db group by batsman) t1\
              inner join \
              (select batsman, count(*) as number_of_times_out from analysis_db where player_dismissed = batsman group by batsman) t2\
              on t1.batsman=t2.batsman) t3\
              inner join\
              (select batsman, count(distinct(match_id)) as total_matches_played \
              from analysis_db group by batsman) t4\
              on t3.batsman = t4.batsman')
f.registerTempTable('finisher_table')

In [ ]:
f = spark.sql('select rank() over (order by finisher desc) as Rank, t1.* \
              from finisher_table t1 \
              inner join \
              no_of_matches_table t2 \
              on t1.batsman = t2.batsman \
              where no_of_matches>9')
f.registerTempTable('finisher_rank')
f.show(10)

In [ ]:
f = spark.sql('select t1.*, (240-rank)/240 as Points from finisher_rank t1')
f.registerTempTable('finisher_points')
f.show(10)

In [ ]:
f = spark.sql('select *, Points*1.25 as Weight from finisher_points')
f.registerTempTable('finisher_weights')
f.show(10)

### Fast Scoring Ability

In [ ]:
# Fast Scoring Ability = Total Runs/Balls Played by Batsman
fsa = spark.sql('select batsman as Batsman, Total_Runs/balls_played as Fast_Scoring_Ability \
                  from (select batsman,sum(batsman_runs) as Total_Runs, count(*) as balls_played \
                  from analysis_db group by batsman)')
fsa.registerTempTable('fast_scoring_ability_table')

In [ ]:
fsa = spark.sql('select rank() over (order by fast_scoring_ability desc) as Rank, t1.* \
                  from fast_scoring_ability_table t1 \
                  inner join \
                  no_of_matches_table t2 \
                  on t1.batsman = t2.batsman where no_of_matches>9')
fsa.registerTempTable('fast_scoring_ability_rank')
fsa.show(10)

In [ ]:
fsa = spark.sql('select t1.*, (240-rank)/240 as Points \
                from fast_scoring_ability_rank t1')
fsa.registerTempTable('fast_scoring_ability_points')
fsa.show(10)

In [ ]:
fsa = spark.sql('select t1.*, Points*1.25 as Weight from fast_scoring_ability_points t1')
fsa.registerTempTable('fast_scoring_ability_weights')
fsa.show(10)

### Consistency

In [ ]:
# Consistency = Total Runs/Number of Times Out
con = spark.sql('select t1.batsman as Batsman,t1.Total_runs/t2.no_of_times_dismissed as Consistency \
                        from (select batsman,sum(batsman_runs) as Total_runs \
                        from analysis_db group by batsman) t1 \
                        inner join \
                        (select batsman, count(*) as no_of_times_dismissed \
                        from analysis_db where player_dismissed is not null \
                        group by batsman) t2 on t1.batsman=t2.batsman')
con.registerTempTable('consistency_table')

In [ ]:
con = spark.sql('select rank() over (order by consistency desc) as Rank, t1.* \
                  from consistency_table t1 \
                  inner join \
                  no_of_matches_table t2 \
                  on t1.batsman = t2.batsman where no_of_matches>9')
con.registerTempTable('consistency_rank')
con.show(10)

In [ ]:
con = spark.sql('select t1.*, (240-rank)/240 as Points from consistency_rank t1')
con.registerTempTable('consistency_points')
con.show(10)

In [ ]:
con = spark.sql('select t1.*, Points as Weight from consistency_points t1')
con.registerTempTable('consistency_weights')
con.show(10)

###  Running Between Wickets

In [ ]:
# Running Between Wickets = (Total Runs – (4*Fours + 6*Sixes))/(Total Balls Played – Boundary Balls)
rbw = spark.sql('select t9.batsman as Batsman, nvl(t8.running_between_wickets,0) as Running_Between_Wickets from \
                (select t4.batsman, t4.first_bracket/t7.second_bracket as Running_Between_Wickets from \
                (select t1.batsman, t3.total_runs-(t1.fours*4 + t2.sixes*6) as first_bracket \
                from (select batsman,count(*) as fours from analysis_db where batsman_runs = 4 \
                group by batsman) t1 \
                inner join \
                (select batsman,count(*) as sixes from analysis_db where batsman_runs = 6 group by batsman) t2 \
                on t1.batsman=t2.batsman \
                inner join \
                (select batsman,sum(batsman_runs) as total_runs from analysis_db group by batsman) t3 \
                on t3.batsman=t1.batsman) t4 \
                inner join\
                (select t5.batsman, t5.total_balls_played-t6.boundry_balls as second_bracket from \
                (select batsman, count(*) as total_balls_played from analysis_db group by batsman) t5 \
                inner join \
                (select batsman, count(*) as boundry_balls from analysis_db where batsman_runs=4 or batsman_runs=6 group by batsman) t6\
                on t5.batsman=t6.batsman) t7 \
                on t4.batsman=t7.batsman) t8 \
                right join \
                no_of_matches_table t9 \
                on t8.batsman = t9.batsman')
rbw.registerTempTable('running_between_wickets_table')

In [ ]:
rbw = spark.sql('select rank() over (order by running_between_wickets desc) as Rank, t1.* \
                  from running_between_wickets_table t1 \
                  inner join \
                  no_of_matches_table t2\
                  on t1.batsman = t2.batsman where no_of_matches>9')
rbw.registerTempTable('running_between_wickets_rank')
rbw.show(10)

In [ ]:
rbw = spark.sql('select t1.*, (240-rank)/240 as Points from running_between_wickets_rank t1')
rbw.registerTempTable('running_between_wickets_points')
rbw.show(10)

In [ ]:
rbw = spark.sql('select t1.*, Points as weight from running_between_wickets_points t1')
rbw.registerTempTable('running_between_wickets_weights')
rbw.show(10)

In [ ]:
# Table Name for each Metric


## Bowling Metrics 

In [7]:
# nmbo: no. of bowlers
# nmb: no. of matches played by a bowler
# eco: economy
# wta: wicket taking ability
# cons: consistency
# cwta: crucial wicket taking ability
# spi: short performance index

In [8]:
# Number of Bowlers
nmbo = spark.sql('Select count(distinct(bowler)) as No_of_Bowlers from analysis_db ')
nmbo.show()

+-------------+
|No_of_Bowlers|
+-------------+
|          405|
+-------------+



In [9]:
# Number of matches played by a bowler
nmb = spark.sql('select bowler as Bowler, count(distinct(match_id)) as No_of_Matches from analysis_db group by bowler')
nmb.registerTempTable('no_of_matches_bowlers')

### Economy 

In [13]:
# Economy = Runs Scored/(Number of balls bowled by bowler/6)
eco = spark.sql('Select bowler as Bowler, round(runs/overs, 5) as Economy \
                from (Select bowler,round(count(*)/6) \
                as overs,sum(total_runs) as runs \
                from analysis_db \
                group by bowler)')
eco.registerTempTable('economy')
eco.show(10)

+--------------+-------+
|        Bowler|Economy|
+--------------+-------+
| Kuldeep Yadav|8.37857|
|    TM Dilshan|    8.0|
|         A Roy|4.66667|
|    KA Pollard|8.38028|
|M Muralitharan|6.68561|
| LA Carseldine|    6.0|
|       J Botha| 6.9322|
|      DR Smith|8.87097|
|Jaskaran Singh|9.15789|
|    A Flintoff|9.63636|
+--------------+-------+
only showing top 10 rows



In [15]:
eco = spark.sql('select row_number() over (order by e.Economy asc) as Rank, e.*,n.No_of_Matches \
                from economy e \
                inner join \
                no_of_matches_bowlers n \
                on e.Bowler = n.Bowler where n.No_of_Matches>9')
eco.registerTempTable('economy')
eco.show(10)

+----+----------------+-------+-------------+
|Rank|          Bowler|Economy|No_of_Matches|
+----+----------------+-------+-------------+
|   1|   Sohail Tanvir|   6.25|           11|
|   2|      A Chandila|6.28205|           12|
|   3|         J Yadav|6.52632|           12|
|   4|      SM Pollock|6.53191|           13|
|   5|        A Kumble|6.64024|           42|
|   6|      GD McGrath|6.65455|           14|
|   7|        DW Steyn|6.66848|           92|
|   8|  M Muralitharan|6.68561|           66|
|   9|RN ten Doeschate|6.71429|           10|
|  10|       RD Chahar|6.71429|           15|
+----+----------------+-------+-------------+
only showing top 10 rows



In [16]:
eco = spark.sql('select *, round((212 - Rank)/212, 5) as Points, (212 - Rank)*1.5/212 as Weight from economy')
eco.registerTempTable('economy')
eco.show(10)

+----+----------------+-------+-------------+-------+--------+
|Rank|          Bowler|Economy|No_of_Matches| Points|  Weight|
+----+----------------+-------+-------------+-------+--------+
|   1|   Sohail Tanvir|   6.25|           11|0.99528|1.492925|
|   2|      A Chandila|6.28205|           12|0.99057|1.485849|
|   3|         J Yadav|6.52632|           12|0.98585|1.478774|
|   4|      SM Pollock|6.53191|           13|0.98113|1.471698|
|   5|        A Kumble|6.64024|           42|0.97642|1.464623|
|   6|      GD McGrath|6.65455|           14| 0.9717|1.457547|
|   7|        DW Steyn|6.66848|           92|0.96698|1.450472|
|   8|  M Muralitharan|6.68561|           66|0.96226|1.443396|
|   9|RN ten Doeschate|6.71429|           10|0.95755|1.436321|
|  10|       RD Chahar|6.71429|           15|0.95283|1.429245|
+----+----------------+-------+-------------+-------+--------+
only showing top 10 rows



### Wicket Taking Ability

In [42]:
# Wicket Taking Ability = Number of balls bowled/Wickets Taken
wta = spark.sql('(Select t1.bowler as Bowler, round(t2.balls/t1.wickets, 5) as Wicket_Taking_Ability from \
                (Select bowler,count(*) as wickets from analysis_db where player_dismissed is not null \
                and (dismissal_kind = \'bowled\' or  dismissal_kind = \'hit wicket\' \
                or  dismissal_kind = \'stumped\' or  dismissal_kind = \'lbw\' \
                or  dismissal_kind = \'caught and bowled\' or  dismissal_kind = \'caught\') \
                group by bowler) t1 \
                inner join \
                (select count(*) as balls,bowler from analysis_db group by bowler)t2 on \
                t1.bowler = t2.bowler)')
wta.registerTempTable('wicket_taking_ability')
wta.show(10)

+--------------+---------------------+
|        Bowler|Wicket_Taking_Ability|
+--------------+---------------------+
|         A Roy|                 15.0|
| Kuldeep Yadav|             21.48718|
|    TM Dilshan|                 55.0|
|       J Botha|                28.36|
|    KA Pollard|             22.78571|
| LA Carseldine|                  7.0|
|M Muralitharan|             24.70313|
|      DR Smith|             21.42308|
|Jaskaran Singh|                 18.5|
|    A Flintoff|                 33.0|
+--------------+---------------------+
only showing top 10 rows



In [43]:
wta = spark.sql('select row_number() over (order by w.Wicket_Taking_Ability asc) as Rank, w.*, n.No_of_Matches \
                from wicket_taking_ability w \
                inner join \
                no_of_matches_bowlers n on \
                w.Bowler = n.Bowler where n.No_of_Matches > 9')
wta.registerTempTable('wicket_taking_ability')
wta.show(10)

+----+--------------+---------------------+-------------+
|Rank|        Bowler|Wicket_Taking_Ability|No_of_Matches|
+----+--------------+---------------------+-------------+
|   1|       A Zampa|             11.84211|           11|
|   2| Sohail Tanvir|             12.04545|           11|
|   3|       K Ahmed|             12.68421|           10|
|   4|        N Rana|             13.42857|           12|
|   5|      K Rabada|                 14.0|           18|
|   6|      BJ Hodge|                 14.0|           20|
|   7|  CRD Fernando|             14.64706|           10|
|   8|    YA Abdulla|                 14.8|           11|
|   9|A Ashish Reddy|                 15.0|           20|
|  10|       S Gopal|             15.60526|           30|
+----+--------------+---------------------+-------------+
only showing top 10 rows



In [45]:
wta = spark.sql('select *,round((212-Rank)/212, 5) as Points,round(1.5*(212-Rank)/212, 5) as Weight \
                from wicket_taking_ability')
wta.registerTempTable('wicket_taking_ability')
wta.show(10)

+----+---------------+---------------------+-------------+-------+-------+
|Rank|         Bowler|Wicket_Taking_Ability|No_of_Matches| Points| Weight|
+----+---------------+---------------------+-------------+-------+-------+
|   1|        A Zampa|             11.84211|           11|0.99528|1.49293|
|   2|  Sohail Tanvir|             12.04545|           11|0.99057|1.48585|
|   3|        K Ahmed|             12.68421|           10|0.98585|1.47877|
|   4|         N Rana|             13.42857|           12|0.98113|1.47170|
|   5|       K Rabada|                 14.0|           18|0.97642|1.46462|
|   6|       BJ Hodge|                 14.0|           20| 0.9717|1.45755|
|   7|   CRD Fernando|             14.64706|           10|0.96698|1.45047|
|   8|     YA Abdulla|                 14.8|           11|0.96226|1.44340|
|   9| A Ashish Reddy|                 15.0|           20|0.95755|1.43632|
|  10|        S Gopal|             15.60526|           30|0.95283|1.42925|
|  11|    Imran Tahir|   

### Consistency 

In [49]:
# Consistency = Runs Conceded/Wickets Taken
cons = spark.sql('select t1.bowler as Bowler, round(t1.runs/t2.wickets, 5) as Consistency \
                        from (select sum(total_runs) as runs,bowler from analysis_db group by bowler) t1 \
                        inner join \
                        (Select bowler,count(*) as wickets from analysis_db where player_dismissed is not null \
                        and (dismissal_kind = \'bowled\' or  dismissal_kind = \'hit wicket\' \
                        or  dismissal_kind = \'stumped\' or  dismissal_kind = \'lbw\' \
                        or  dismissal_kind = \'caught and bowled\' or  dismissal_kind = \'caught\') \
                        group by bowler)t2 on t1.bowler = t2.bowler')
cons.registerTempTable('consistency')
cons.show(10)

+--------------+-----------+
|        Bowler|Consistency|
+--------------+-----------+
|         A Roy|       14.0|
| Kuldeep Yadav|   30.07692|
|    TM Dilshan|       73.6|
|       J Botha|      32.72|
|    KA Pollard|     31.875|
| LA Carseldine|        6.0|
|M Muralitharan|   27.57813|
|      DR Smith|   31.73077|
|Jaskaran Singh|       29.0|
|    A Flintoff|       53.0|
+--------------+-----------+
only showing top 10 rows



In [50]:
cons = spark.sql('select row_number() over (order by c.Consistency asc) as Rank, c.*,n.No_of_Matches \
                    from consistency c \
                    inner join \
                    no_of_matches_bowlers n on \
                    c.Bowler = n.Bowler where n.No_of_Matches > 9')
cons.registerTempTable('consistency')
cons.show(10)

+----+--------------+-----------+-------------+
|Rank|        Bowler|Consistency|No_of_Matches|
+----+--------------+-----------+-------------+
|   1| Sohail Tanvir|       12.5|           11|
|   2|       A Zampa|   14.78947|           11|
|   3|        N Rana|   17.71429|           12|
|   4|  CRD Fernando|       18.0|           10|
|   5|      BJ Hodge|   18.23529|           20|
|   6|       K Ahmed|   18.47368|           10|
|   7|AD Mascarenhas|   19.21053|           13|
|   8|      K Rabada|   19.32258|           18|
|   9|  DE Bollinger|   19.35135|           27|
|  10|   MF Maharoof|    19.7037|           20|
+----+--------------+-----------+-------------+
only showing top 10 rows



In [51]:
cons = spark.sql('select *,round((212-Rank)/212, 5) as Points, round((212-Rank)/212, 5) as Weights from consistency')
cons.registerTempTable('consistency')
cons.show(10)

+----+--------------+-----------+-------------+-------+-------+
|Rank|        Bowler|Consistency|No_of_Matches| Points|Weights|
+----+--------------+-----------+-------------+-------+-------+
|   1| Sohail Tanvir|       12.5|           11|0.99528|0.99528|
|   2|       A Zampa|   14.78947|           11|0.99057|0.99057|
|   3|        N Rana|   17.71429|           12|0.98585|0.98585|
|   4|  CRD Fernando|       18.0|           10|0.98113|0.98113|
|   5|      BJ Hodge|   18.23529|           20|0.97642|0.97642|
|   6|       K Ahmed|   18.47368|           10| 0.9717| 0.9717|
|   7|AD Mascarenhas|   19.21053|           13|0.96698|0.96698|
|   8|      K Rabada|   19.32258|           18|0.96226|0.96226|
|   9|  DE Bollinger|   19.35135|           27|0.95755|0.95755|
|  10|   MF Maharoof|    19.7037|           20|0.95283|0.95283|
+----+--------------+-----------+-------------+-------+-------+
only showing top 10 rows



### Crucial Wicket Taking Ability

In [62]:
# Crucial Wicket Taking Ability = Number of times Four or Five Wickets Taken/Number of Innings Played
cwta = spark.sql('select t2.bowler as Bowler, round(nvl(t1.no_of_4wickets/t2.innings,0), 5) as Crucial_Wicket_Taking_Ablity \
                 from (select bowler,count(*) as no_of_4wickets from (select * from \
                 (select match_id,bowler,count(*) as wickets from analysis_db where player_dismissed \
                 is not null \
                 and (dismissal_kind = \'bowled\' or  dismissal_kind = \'hit wicket\' \
                 or  dismissal_kind = \'stumped\' or  dismissal_kind = \'lbw\' \
                 or  dismissal_kind = \'caught and bowled\' or  dismissal_kind = \'caught\') \
                 group by bowler,match_id ) \
                 where wickets > 3) group by bowler)t1 \
                 right join \
                 (select bowler,count(match_id) as \
                 innings from (select distinct(match_id),bowler from analysis_db) \
                 group by bowler)t2 \
                 on t1.bowler = t2.bowler order by Crucial_Wicket_Taking_Ablity desc')
cwta.registerTempTable('crucial_wicket_taking_ablity')
cwta.show(10)

+---------------+----------------------------+
|         Bowler|Crucial_Wicket_Taking_Ablity|
+---------------+----------------------------+
|       A Joseph|                     0.33333|
|  Shoaib Akhtar|                     0.33333|
|  Sohail Tanvir|                     0.18182|
|     YA Abdulla|                     0.18182|
|       Umar Gul|                     0.16667|
|         AJ Tye|                     0.15385|
|        L Ngidi|                     0.14286|
|Karanveer Singh|                     0.11111|
|       S Curran|                     0.11111|
|       K Rabada|                     0.11111|
+---------------+----------------------------+
only showing top 10 rows



In [63]:
cwta = spark.sql('select rank() over (order by cw.Crucial_Wicket_Taking_Ablity desc) as Rank, cw.*,n.No_of_Matches \
                 from crucial_wicket_taking_ablity cw \
                 inner join no_of_matches_bowlers n on \
                 cw.Bowler = n.Bowler where n.No_of_Matches > 9')
cwta.registerTempTable('crucial_wicket_taking_ablity')
cwta.show(10)

+----+-------------+----------------------------+-------------+
|Rank|       Bowler|Crucial_Wicket_Taking_Ablity|No_of_Matches|
+----+-------------+----------------------------+-------------+
|   1|Sohail Tanvir|                     0.18182|           11|
|   1|   YA Abdulla|                     0.18182|           11|
|   3|       AJ Tye|                     0.15385|           26|
|   4|     K Rabada|                     0.11111|           18|
|   5|     J Theron|                         0.1|           10|
|   5|  PC Valthaty|                         0.1|           10|
|   5| CRD Fernando|                         0.1|           10|
|   8|      A Zampa|                     0.09091|           11|
|   8|    CJ Jordan|                     0.09091|           11|
|  10|   A Chandila|                     0.08333|           12|
+----+-------------+----------------------------+-------------+
only showing top 10 rows



In [64]:
cwta = spark.sql('select *,round((212-Rank)/212, 5) as Points, round(1.5*(212-Rank)/212, 5) as Weights \
                 from crucial_wicket_taking_ablity')
cwta.registerTempTable('crucial_wicket_taking_ablity')
cwta.show(10)

+----+-------------+----------------------------+-------------+-------+-------+
|Rank|       Bowler|Crucial_Wicket_Taking_Ablity|No_of_Matches| Points|Weights|
+----+-------------+----------------------------+-------------+-------+-------+
|   1|Sohail Tanvir|                     0.18182|           11|0.99528|1.49293|
|   1|   YA Abdulla|                     0.18182|           11|0.99528|1.49293|
|   3|       AJ Tye|                     0.15385|           26|0.98585|1.47877|
|   4|     K Rabada|                     0.11111|           18|0.98113|1.47170|
|   5|     J Theron|                         0.1|           10|0.97642|1.46462|
|   5|  PC Valthaty|                         0.1|           10|0.97642|1.46462|
|   5| CRD Fernando|                         0.1|           10|0.97642|1.46462|
|   8|      A Zampa|                     0.09091|           11|0.96226|1.44340|
|   8|    CJ Jordan|                     0.09091|           11|0.96226|1.44340|
|  10|   A Chandila|                    

### Short Performance Index

In [71]:
# Short Performance Index = (Wickets Taken – 4* Number of Times Four Wickets Taken – 5* Number of Times Five Wickets Taken)/(Innings Played – Number of Times Four Wickets or Five Wickets Taken)
spi = spark.sql('select n.bowler as Bowler, round(nvl(t5.Short_Performance_Index,0), 5) as Short_Performance_Index \
                from (select t1.bowler,(t3.wickets - 4*t1.no_of_4wickets - 5*t2.no_of_4wickets)/ \
                (t4.innings - t1.no_of_4wickets - t2.no_of_4wickets) as Short_Performance_Index \
                from (select bowler,count(*) as no_of_4wickets \
                from (select * from (select match_id,bowler,count(*) as wickets from analysis_db where player_dismissed \
                is not null \
                and (dismissal_kind = \'bowled\' or  dismissal_kind = \'hit wicket\' \
                or  dismissal_kind = \'stumped\' or  dismissal_kind = \'lbw\' \
                or  dismissal_kind = \'caught and bowled\' or  dismissal_kind = \'caught\')\
                group by bowler, match_id ) \
                where wickets = 4) group by bowler) t1 \
                inner join \
                (select bowler,count(*) as no_of_4wickets from (select * from \
                (select match_id,bowler,count(*) as wickets from analysis_db where player_dismissed \
                is not null \
                and (dismissal_kind = \'bowled\' or  dismissal_kind = \'hit wicket\' \
                or dismissal_kind = \'stumped\' or  dismissal_kind = \'lbw\' \
                or dismissal_kind = \'caught and bowled\' or  dismissal_kind = \'caught\')\
                group by bowler,match_id ) \
                where wickets = 5) group by bowler) t2 \
                inner join \
                (select bowler,count(*) as wickets from analysis_db where player_dismissed is not null \
                and (dismissal_kind = \'bowled\' or  dismissal_kind = \'hit wicket\' \
                or  dismissal_kind = \'stumped\' or  dismissal_kind = \'lbw\' \
                or  dismissal_kind = \'caught and bowled\' or  dismissal_kind = \'caught\') \
                group by bowler) t3 \
                inner join \
                (select bowler,count(match_id) as \
                innings from (select distinct(match_id),bowler from analysis_db) group by bowler) t4 \
                on t1.bowler = t2.bowler and t1.bowler = t3.bowler and t1.bowler = t4.bowler) t5 \
                right join \
                no_of_matches_bowlers n on t5.Bowler = n.Bowler order by Short_Performance_Index desc')
spi.registerTempTable('short_performance_index')
spi.show(10)

+---------------+-----------------------+
|         Bowler|Short_Performance_Index|
+---------------+-----------------------+
|     SL Malinga|                1.22609|
|        B Kumar|                1.05263|
|       MM Patel|                1.01667|
|         AJ Tye|                    1.0|
|       A Mishra|                0.97203|
|      SP Narine|                0.91176|
|Harbhajan Singh|                0.90968|
|       L Balaji|                0.85507|
|    JP Faulkner|                0.82456|
|       A Kumble|                0.82051|
+---------------+-----------------------+
only showing top 10 rows



In [76]:
spi = spark.sql('select rank() over (order by sp.Short_Performance_Index desc) as Rank, sp.*, n.No_of_Matches \
                from short_performance_index sp \
                inner join \
                no_of_matches_bowlers n on \
                sp.Bowler = n.Bowler where n.No_of_Matches > 9')
spi.registerTempTable('short_performance_index')
spi.show(10)

+----+---------------+-----------------------+-------------+
|Rank|         Bowler|Short_Performance_Index|No_of_Matches|
+----+---------------+-----------------------+-------------+
|   1|     SL Malinga|                1.22609|          122|
|   2|        B Kumar|                1.05263|          117|
|   3|       MM Patel|                1.01667|           63|
|   4|         AJ Tye|                    1.0|           26|
|   5|       A Mishra|                0.97203|          147|
|   6|      SP Narine|                0.91176|          109|
|   7|Harbhajan Singh|                0.90968|          157|
|   8|       L Balaji|                0.85507|           73|
|   9|    JP Faulkner|                0.82456|           60|
|  10|       A Kumble|                0.82051|           42|
+----+---------------+-----------------------+-------------+
only showing top 10 rows



In [78]:
spi = spark.sql('select *,round((212-Rank)/212, 5) as Points, round((212-Rank)/212, 5) as Weights \
                from short_performance_index')
spi.registerTempTable('short_performance_index')
spi.show(10)

+----+---------------+-----------------------+-------------+-------+-------+
|Rank|         Bowler|Short_Performance_Index|No_of_Matches| Points|Weights|
+----+---------------+-----------------------+-------------+-------+-------+
|   1|     SL Malinga|                1.22609|          122|0.99528|0.99528|
|   2|        B Kumar|                1.05263|          117|0.99057|0.99057|
|   3|       MM Patel|                1.01667|           63|0.98585|0.98585|
|   4|         AJ Tye|                    1.0|           26|0.98113|0.98113|
|   5|       A Mishra|                0.97203|          147|0.97642|0.97642|
|   6|      SP Narine|                0.91176|          109| 0.9717| 0.9717|
|   7|Harbhajan Singh|                0.90968|          157|0.96698|0.96698|
|   8|       L Balaji|                0.85507|           73|0.96226|0.96226|
|   9|    JP Faulkner|                0.82456|           60|0.95755|0.95755|
|  10|       A Kumble|                0.82051|           42|0.95283|0.95283|

In [79]:
# Table Name for each Metric
